# ROOM - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu lokalnej bazy danych **ROOM** w aplikacji.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExam82azljYWxraXAwcDM3dWk1NnowMTYwbDFqY3lzcDB1bm1uZHQxNCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/wJ4PVlB2HlfUf2giVm/giphy.gif" width="200" />

`Room` jest to biblioteka dostarczona przez Android Jetpack, która ułatwia pracę z lokalną bazą danych `SQLite`. Stanowi wygodny i wydajny sposób na przechowywanie danych wewnętrznych aplikacji. `SQLite` to lekka, wbudowana baza danych, która jest szeroko stosowana w aplikacjach mobilnych i innych systemach osadzonych.

Główne cechy `SQLite` to:
- Działa jako biblioteka dostępna w postaci pliku w aplikacji, co oznacza, że nie wymaga oddzielnego procesu serwera bazy danych. Aplikacja może bezpośrednio komunikować się z bazą danych.
- Jest samowystarczalna, co oznacza, że cała baza danych jest przechowywana w jednym pliku. Nie ma potrzeby konfigurowania lub zarządzania wieloma plikami lub zasobami.
- Transakcje - Obsługuje transakcje, co umożliwia grupowanie operacji bazodanowych jako pojedyncze, atomowe działanie. Transakcje są ważne, gdy chodzi o utrzymanie integralności danych. Każda operacja na bazie jest wykonywana w całości lub w ogóle - jest to ważne przy modyfikowaniu wielu tabel, oraz asynchronicznym przetwarzaniu.
- Wsparcie dla standardowego `SQL` - obsługuje większość standardowego języka `SQL`, co ułatwia programowanie i wykonywanie zapytań.
- Jest zaprojektowany w taki sposób, aby działał wydajnie nawet na urządzeniach o ograniczonych zasobach sprzętowych.

Główne składniki biblioteki `Room` to:
- `Entity` - Reprezentuje tabelę w bazie danych `SQLite`. Każda klasa oznaczona adnotacją `@Entity` może być mapowana do jednej tabeli w bazie danych, a pola klasy odpowiadają kolumnom tej tabeli.
- `DAO` (*Data Access Object*) - Definiuje metody, które umożliwiają dostęp do danych w bazie danych. Możemy zdefiniować interfejs `DAO` za pomocą adnotacji `@Dao`, a Room automatycznie dostarczy implementację tych metod.
- `Database` - Klasa bazowa, która reprezentuje bazę danych. To miejsce, gdzie definiujemy wszystkie *encje*, które mają zostać użyte w aplikacji, oraz wersję bazy danych. Room tworzy implementację bazy danych opartej na tej klasie.

Aplikacja będzie wyświetlać listę wszystkich użytkowników przechowywanych w bazie danych na urządzeniu. Dodamy operacje dodawania pojedynczego użytkownika oraz czyszczenia danych z bazy.

Dodajmy niezbędne zależności i pluginy do plików konfiguracyjnych aplikacji

```kotlin
// ROOM
implementation ("androidx.room:room-runtime:2.5.2")
annotationProcessor ("androidx.room:room-compiler:2.5.2")

// ViewModel
implementation ("androidx.lifecycle:lifecycle-viewmodel:2.5.1")
// LiveData
implementation ("androidx.lifecycle:lifecycle-livedata:2.5.1")
```

Będziemy dodawać użytkowników, więc dodajmy obiekt pomocniczy, który ułatwi ich generowanie.

In [ ]:
public class User {
    private String firstName;
    private String lastName;

    // Constructor
    public User(String firstName, String lastName) {
        this.firstName = firstName;
        this.lastName = lastName;
    }

    public void setId(@NonNull int id) {
        this.id = id;
    }

    public String getFirstName() {
        return firstName;
    }

    public void setFirstName(String firstName) {
        this.firstName = firstName;
    }

    public String getLastName() {
        return lastName;
    }

    public void setLastName(String lastName) {
        this.lastName = lastName;
    }

    // hashCode and equals methods
    @Override
    public int hashCode() {
        final int prime = 31;
        int result = 1;
        result = prime * result + id;
        result = prime * result + ((firstName == null) ? 0 : firstName.hashCode());
        result = prime * result + ((lastName == null) ? 0 : lastName.hashCode());
        return result;
    }

    @Override
    public boolean equals(Object obj) {
        if (this == obj) return true;
        if (obj == null) return false;
        if (getClass() != obj.getClass()) return false;

        User other = (User) obj;

        if (id != other.id) return false;
        if (firstName == null)
            if (other.firstName != null)
                return false;
        else return false;
        if (lastName == null) return other.lastName == null;
        else return lastName.equals(other.lastName);
    }
}

In [ ]:
public final class DataProvider {
    private DataProvider(){}
    private static final List<String> firstNames = Arrays.asList(
            "Adam", "Ewa", "Jan", "Anna", "Piotr", "Maria", "Tomasz", "Małgorzata", "Krzysztof", "Alicja",
            "Andrzej", "Joanna", "Michał", "Barbara", "Kamil", "Magdalena", "Robert", "Monika", "Mateusz", "Natalia"
    );

    private static final List<String> lastNames = Arrays.asList(
            "Nowak", "Kowalski", "Wiśniewski", "Wójcik", "Kowalczyk", "Kamiński", "Lewandowski", "Zieliński", "Szymański",
            "Woźniak", "Dąbrowski", "Kozłowski", "Jankowski", "Mazur", "Kwiatkowski", "Krawczyk", "Piotrowski", "Grabowski",
            "Nowakowski", "Pawłowski"
    );

    public static User getUser() {
        Random random = new Random();
        String firstName = firstNames.get(random.nextInt(firstNames.size()));
        String lastName = lastNames.get(random.nextInt(lastNames.size()));
        return new User(firstName, lastName);
    }
}

## Elementy Room

Przejdźmy do dodania wszystkich elementów bazy danych, rozpoczniemi od `Entity`. Zmodyfikujmy wcześniej dodaną klasę `User`.

In [ ]:
@Entity(tableName = "user_table")
public class User {
    @PrimaryKey(autoGenerate = true)
    private int id;

    private String firstName;
    private String lastName;

    // Constructor
    public User(String firstName, String lastName) {
        this.firstName = firstName;
        this.lastName = lastName;
    }

    // Getters and Setters
    @NonNull
    public int getId() {
        return id;
    }

    public void setId(@NonNull int id) {
        this.id = id;
    }

    public String getFirstName() {
        return firstName;
    }

    public void setFirstName(String firstName) {
        this.firstName = firstName;
    }

    public String getLastName() {
        return lastName;
    }

    public void setLastName(String lastName) {
        this.lastName = lastName;
    }

    // hashCode and equals methods
    @Override
    public int hashCode() {
        final int prime = 31;
        int result = 1;
        result = prime * result + id;
        result = prime * result + ((firstName == null) ? 0 : firstName.hashCode());
        result = prime * result + ((lastName == null) ? 0 : lastName.hashCode());
        return result;
    }

    @Override
    public boolean equals(Object obj) {
        if (this == obj) return true;
        if (obj == null) return false;
        if (getClass() != obj.getClass()) return false;

        User other = (User) obj;

        if (id != other.id) return false;
        if (firstName == null)
            if (other.firstName != null)
                return false;
        else return false;
        if (lastName == null) return other.lastName == null;
        else return lastName.equals(other.lastName);
    }
}

- `@Entity(tableName = "user_table")` - Adnotacja `@Entity` jest używana do oznaczenia klasy jako *encji* bazy danych. Oznacza to, że obiekty tej klasy będą reprezentować wiersze w tabeli bazy danych. `tableName` to atrybut adnotacji `@Entity`, który definiuje nazwę tabeli, do której będą mapowane obiekty tej klasy. W tym przypadku tabela będzie miała nazwę `"user_table"`.
- `    @PrimaryKey(autoGenerate = true) private int id;` - Adnotacja `@PrimaryKey` informuje, że pole `id` będzie kluczem głównym tabeli. W bazie danych, pole oznaczone jako klucz główny musi być unikatowe dla każdego wiersza. `autoGenerate = true` oznacza, że wartość klucza będzie generowana automatycznie przy dodawaniu nowego rekordu do tabeli.

Adnotacja `@Entity` może posiadać szereg innych atrybutów:
- `indices` - Pozwala na zdefiniowanie indeksów dla jednego lub wielu pól w tabeli. Indeksy pomagają w przyspieszeniu wyszukiwania danych w bazie danych.
```kotlin
@Entity(tableName = "user_table", indices = [Index(value = ["firstName", "lastName"])])
public class User(
    // ...
)
```
- `foreignKeys` - Pozwala na zdefiniowanie kluczy obcych w tabeli. Określa relacje między tabelą bieżącą a innymi tabelami w bazie danych.
```kotlin
@Entity(
    tableName = "order_table",
    foreignKeys = [
        ForeignKey(
            entity = User::class,
            parentColumns = ["id"],
            childColumns = ["user_id"],
            onDelete = ForeignKey.CASCADE
        )
    ]
)
public class Order(
    // ...
    val user_id: Int
)
```
- `primaryKeys` - Pozwala na zdefiniowanie niestandardowego zestawu pól jako kluczy głównych tabeli.
```kotlin
@Entity(tableName = "book_table", primaryKeys = ["isbn", "title"])
public class Book(
    val isbn: String,
    val title: String,
    // ...
)
```
- `ignoredColumns` - Pozwala na zdefiniowanie pól, które będą ignorowane przez `Room` i nie będą mapowane do tabeli w bazie danych.
```kotlin
@Entity(tableName = "user_table", ignoredColumns = ["age"])
public class User(
    val firstName: String,
    val lastName: String,
    val age: Int // ignorowane
    // ...
)
```

Kolejnym elementem będzie dodanie interfejsu `DAO`

In [ ]:
@Dao
public interface UserDao {

    @Query("SELECT * FROM user_table ORDER BY lastName ASC, firstName ASC")
    LiveData<List<User>> getUsers();

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    void insert(User user);

    @Query("DELETE FROM user_table")
    void deleteAll();
}

Interfejs `DAO` zawiera deklaracje metod, które pozwalają na interakcję z bazą danych i wykonywanie operacji.

- `@Query("SELECT * FROM user_table ORDER BY lastName ASC, firstName ASC")` - Jest to metoda do pobierania użytkowników z bazy danych. Zapytanie `SQL` `SELECT * FROM user_table ORDER BY lastName ASC, firstName ASC` wybiera wszystkie kolumny z tabeli `"user_table"` i sortuje wyniki według kolumny `lastName` rosnąco, a następnie według kolumny `firstName` rosnąco. Ta metoda zwraca dane jako `LiveData<List<User>>`, co oznacza, że może przesyłać nowe wyniki w czasie rzeczywistym (np. gdy dane się zmienią).
- `@Insert(onConflict = OnConflictStrategy.IGNORE)` - Jest to metoda do dodawania użytkownika do bazy danych. Adnotacja `@Insert` oznacza, że ta metoda jest używana do wstawiania danych do tabeli. Atrybut `onConflict = OnConflictStrategy.IGNORE` oznacza, że jeśli próba wstawienia użytkownika zakończy się konfliktem (np. duplikatem klucza głównego), zostanie zignorowana.
- `@Query("DELETE FROM user_table")` - Jest to metoda do usuwania wszystkich użytkowników z bazy danych. Zapytanie `SQL` `DELETE FROM user_table` usuwa wszystkie wiersze z tabeli `"user_table"` i czyszczenie jej zawartości.

- `@Insert` - specjalna adnotacja, która nie wymaga podawania wprost kodu `SQL` - istanieją jeszcze `@Delete` oraz `@Update` - mocno ułatwia to wykonywanie podstawowych operacji na bazie. Adnotacja `@Query` wymaga podania pełnego zapytania `SQL` jako `String`

Lista niektórych adnotacji, które można wykorzystać:
- `@Update` - służy do aktualizacji danych w bazie danych. Metody oznaczone tą adnotacją muszą przyjmować jako argumenty obiekty klas, które reprezentują wiersze w tabeli. `Room` automatycznie wygeneruje odpowiednie zapytania `SQL` do zaktualizowania tych danych.
```kotlin
@Dao
public interface UserDao {
    // ...

    @Update
    void update(User user)

    // ...
}
```
- `@Delete` - jest używana do usuwania danych z bazy danych. Metody oznaczone tą adnotacją powinny przyjmować jako argumenty obiekty klas reprezentujących wiersze, które mają zostać usunięte.
```kotlin
@Dao
public interface UserDao {
    // ...

    @Delete
    void delete(User user)

    // ...
}
```
- `@Query` - pozwala na definiowanie własnych zapytań `SQL`.
```kotlin
@Dao
public interface UserDao {
    // ...

    @Query("SELECT * FROM user_table WHERE firstName = :firstName") // zwraca listę wszystkich użytkowników gdzie pole w bazie (firstName) 
                                                                    // jest równe argumentowi przekazanemu w funkcji (:firstName)
    List<User> getUsersByFirstName(String firstName)

    // ...
}
```
- `@RawQuery` - pozwala na wykonywanie niezdefiniowanych z góry zapytań `SQL`, które można przekazać jako argument typu `SupportSQLiteQuery`.
```kotlin
@Dao
public interface UserDao {
    // ...

    @RawQuery
    List<User> getUsersByRawQuery(SupportSQLiteQuery query)

    // ...
}
```
- `@Transaction` - używana jest do oznaczenia metod, które wymagają wykonania kilku metod w transakcji. Transakcje pozwalają na wykonywanie zestawu operacji jako pojedynczą, atomową operację - albo transakcja jest wykonana w pełni, albo wcale.
```kotlin
@Dao
public interface UserDao {
    // ...

    @Transaction
    void insertAndUpdate(User user, User updatedUser) {
        insert(user)
        update(updatedUser)
    }

    // ...
}
```

Nie implementujemy metod tego interfejsu, ponieważ to `Room` automatycznie generuje implementacje tych metod w czasie kompilacji. Koncepcja ta nazywa się *automatyczną implementacją*.

`Room` wykorzystuje mechanizm **Proxy** w języku Kotlin (lub **Refleksję** w języku Java), aby analizować interfejs `DAO` i generować kod `SQL` oraz operacje na bazie danych zgodnie z adnotacjami i deklaracjami metod. Dzięki temu nie musimy ręcznie implementować tych metod i zapytań.

Ostatnim elementem będzie utworzenie klasy bazowej.

In [ ]:
@Database(entities = {User.class}, version = 1, exportSchema = false)
public abstract class UserDatabase extends RoomDatabase {

    public abstract UserDao userDao();

    private static volatile UserDatabase INSTANCE;

    public static UserDatabase getDatabase(final Context context) {
        if (INSTANCE == null) {
            synchronized (UserDatabase.class) {
                if (INSTANCE == null) {
                    INSTANCE = Room.databaseBuilder(context.getApplicationContext(),
                                    UserDatabase.class, "user_database_java")
                            .build();
                }
            }
        }
        return INSTANCE;
    }
}


Klasa ta reprezentuje bazę danych dla aplikacji, która zawiera tylko jedną tabelę `User`. Wykorzystuje wzorzec **Singleton** i synchronizację w celu zapewnienia, że baza danych będzie miała tylko jedną instancję w całej aplikacji.

- `@Database(entities = {User.class}, version = 1, exportSchema = false)` - Adnotacja `@Database` jest używana do oznaczenia klasy `UserDatabase` jako bazowej klasy bazy danych `Room`.
    - `entities` definiuje tablicę klas encji, które będą zawarte w tej bazie danych. 
    - `version` definiuje numer wersji bazy danych.
    - `exportSchema` określa, czy `Room` ma eksportować schemat bazy danych do pliku na urządzeniu. W tym przypadku jest ustawiony na false - migracje bazy są poza zakresem przedmiotu.
- `public abstract UserDao userDao()` - Jest to deklaracja abstrakcyjnej metody, która zwraca obiekt o typie interfejsu `UserDao`. Ta metoda będzie używana do uzyskania dostępu do operacji bazodanowych związanych z encją `User`. Jest metodą abstrakcyjną, ponieważ nie ma zdefiniowanej implementacji. Faktyczna implementacja tej metody jest generowana automatycznie przez `Room` w czasie kompilacji.
- `private static volatile UserDatabase INSTANCE` - Zmienna `Instance` jest oznaczona jako `volatile`, co zapewnia, że jej wartość jest zawsze widoczna dla innych wątków.
- `public static UserDatabase getDatabase(final Context context)` - Jest to metoda fabryczna, która tworzy lub zwraca istniejącą instancję bazy danych. Jeśli instancja bazy danych już istnieje, zostanie zwrócona; w przeciwnym razie zostanie utworzona za pomocą `Room.databaseBuilder()`.
- `Room.databaseBuilder()` używa `context` aplikacji, klasy `UserDatabase` oraz nazwy bazy danych (`"user_database"`) do skonfigurowania i zbudowania instancji `UserDatabase`. 
- `Metoda synchronized()` jest używana do synchronizacji dostępu do kodu, co zapewnia, że tylko jeden wątek może uzyskać dostęp do tej sekcji jednocześnie.

## ViewModel

Przejdźmy do utworzenia klasy `UserViewModel`, ponieważ `UserDatabase` wymaga podania kontekstu aplikacji, przekażemy go przez konstruktor. Jest to problem, ponieważ `ViewModel` nie przyjmuje żadnych parametrów. Mamy do dyspozycji również `AndroidViewModel`, który umożliwia przekazanie kontekstu jako parametru do instancji `ViewModel`.

Operacje na bazie danych domyślnie muszą być wykonane asynchronicznie, potrzebujemy więc odpowiednich metod. Wykorzystamy klasy `Executor` i `Executors`

In [ ]:
public class AppExecutors {
    private static final Object LOCK = new Object();
    private static AppExecutors instance;
    private final Executor dbIO;

    private AppExecutors(Executor dbIO) {
        this.dbIO = dbIO;
    }

    public static AppExecutors getInstance() {
        if (instance == null) {
            synchronized (LOCK) {
                instance = new AppExecutors(Executors.newSingleThreadExecutor());
            }
        }
        return instance;
    }

    public Executor diskIO() {
        return dbIO;
    }
}

Klasa zawiera implementację wzorców **Singleton** i **Factory** do obsługi wielowątkowego przetwarzania w aplikacji. Zapewnia instancję jednego wątku tła (`dbIO`), który może być używany do wykonywania operacji bazodanowych.

- `private static final Object LOCK = new Object();` - Jest to obiekt `LOCK` używany do synchronizacji przy tworzeniu instancji `AppExecutors`. Synchronizacja jest używana, aby zapewnić, że tylko jeden wątek może tworzyć instancję w danym momencie.
- `private static AppExecutors instance;` - Jest to statyczne pole, które przechowuje jedyną instancję. Dzięki temu, klasa będzie miała tylko jedną instancję w całej aplikacji.
- `private final Executor dbIO;` - Jest to pole przechowujące instancję `Executor`, która reprezentuje pojedynczy wątek tła.
- `private AppExecutors(Executor dbIO)` - Konstruktor jest prywatny, co oznacza, że nie można utworzyć instancji spoza klasy.
- `public static AppExecutors getInstance()` - Jest to metoda, która zwraca jedyną instancję klasy. Wywołanie tej metody pozwala na uzyskanie dostępu do tego samego obiektu w różnych częściach aplikacji.

Dodajmy właściwy `ViewModel`

In [ ]:
public class UserViewModel extends AndroidViewModel {
    private final UserRepository repository;
    private LiveData<List<User>> users;

    public UserViewModel(@NonNull Application application) {
        super(application); // przekazujemy application do AndroidViewModel
        UserDatabase db = UserDatabase.getDatabase(application);
        UserDao dao = db.userDao();
        repository = new UserRepository(dao);

        fetchUsers();
    }

    private void fetchUsers() {
        users = repository.getUsers();
    }

    public LiveData<List<User>> getUsers() {
        return users;
    }

    public void clearUsers() {
        AppExecutors.getInstance().dbIO().execute(repository::clear);
    }

    public void addUser(User user) {
        AppExecutors.getInstance().dbIO().execute(() -> repository.add(user));
    }
}

### **UWAGA!!!**
Tego typu tworzenie obiektów `UserDatabase` i `UserRepository` jest problematyczne - w ostatnim module zapoznamy się z techniką wstrzykiwania zależności (*dependency injection*), która pozwoli nam rozwiązać ten problem.

## ui

Dodajmy Fragment z `RecyclerView` w celu wyświetlenia całej listy.

In [ ]:
public class UserViewHolder extends RecyclerView.ViewHolder {
    private RvItemBinding binding;

    public UserViewHolder(RvItemBinding binding) {
        super(binding.getRoot());
        this.binding = binding;
    }

    public void bind(User item) {
        binding.textView.setText(item.getFirstName() + " " + item.getLastName());
    }
}

In [ ]:
public class UserComparator extends DiffUtil.ItemCallback<User> {
    @Override
    public boolean areItemsTheSame(@NonNull User oldItem, @NonNull User newItem) {
        return oldItem == newItem;
    }

    @Override
    public boolean areContentsTheSame(User oldItem, @NonNull User newItem) {
        return oldItem.equals(newItem);
    }
}

In [ ]:
public class UserAdapter extends ListAdapter<User, UserViewHolder> {
    public UserAdapter(@NonNull DiffUtil.ItemCallback<User> diffCallback) {
        super(diffCallback);
    }

    @NonNull
    @Override
    public UserViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new UserViewHolder(
                RvItemBinding.inflate(
                        LayoutInflater.from(parent.getContext()), parent, false
                )
        );
    }

    @Override
    public void onBindViewHolder(@NonNull UserViewHolder holder, int position) {
        User item = getItem(position);
        holder.bind(item);
    }
}


In [ ]:
public class UserFragment extends Fragment {

    @Nullable
    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, @Nullable ViewGroup container, @Nullable Bundle savedInstanceState) {
        com.example.roombasicsjava.databinding.FragmentUserBinding binding = FragmentUserBinding.inflate(inflater);

        UserViewModel userViewModel = new ViewModelProvider(this).get(UserViewModel.class);

        final UserAdapter userAdapter = new UserAdapter(new UserComparator());
        binding.rvList.setAdapter(userAdapter);
        binding.rvList.setLayoutManager(new LinearLayoutManager(requireActivity()));
        binding.rvList.setItemAnimator(null);
        userViewModel.getUsers().observe(requireActivity(), userAdapter::submitList);

        binding.addButton.setOnClickListener(v -> {userViewModel.addUser(DataProvider.getUser());});
        binding.clearButton.setOnClickListener(v -> userViewModel.clearUsers());


        return binding.getRoot();
    }
}

Ustawiamy `itemAnimator` na `null`, aby uniknąć wyświetlania domyślnej animacji (*fadeout-fadein*) przy przeładowaniu listy.

Możemy przetestować aplikacjię.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExam82azljYWxraXAwcDM3dWk1NnowMTYwbDFqY3lzcDB1bm1uZHQxNCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/wJ4PVlB2HlfUf2giVm/giphy.gif" width="200" />